In [4]:
# In case of emergency
# %pip install pysatSpaceWeather
# %pip install pysatNASA
# Start with basic imports
import datetime as dt
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pysat
import pysatNASA
import pysatSpaceWeather as py_sw

from math import isnan
import kamodo_ccmc.flythrough.model_wrapper as MW
import kamodo_ccmc.tools.plotfunctions as pf
from kamodo_ccmc.tools.plotfunctions import toColor, toLog10
from kamodo_ccmc.flythrough import SatelliteFlythrough as SF
from kamodo_ccmc.tools.functionalize import Functionalize_Dataset as FD
import pytz
from datetime import datetime

In [5]:
# Set data directory if user hasn't already set one
print(f"old: {pysat.params['data_dirs']}")
if len(pysat.params['data_dirs']) == 0 or pysat.params['data_dirs'] == ['.']:
    # Set a directory for pysat to use for data
    pysat.params['data_dirs'] = '/home/jovyan/scratch_space/.pysat/'
else:
    print('pysat directory has been set previously. Leaving unchanged.')

print(f"new: {pysat.params['data_dirs']}")

old: ['.']
new: ['/home/jovyan/scratch_space/.pysat']


In [6]:
# If you get an error about the data directory not existing yet, do one of these then repeat the cell above
#pysat.params['data_dirs'] = '/home/jovyan/scratch_space/.pysat/'
#pysat.params['data_dirs'] = '/home/jovyan/.pysat/'

In [7]:
# Register the COSMIC 2 instrument from pysatCDAAC with pysat
pysat.utils.registry.register_by_module(pysatNASA.instruments)

In [8]:
# Dates during a magnetic storm with Kp greater than 6.
start = dt.datetime(2023, 3, 22)
stop = dt.datetime(2023, 3, 26)

print('Data start DOY =',start.timetuple().tm_yday)
print('Data end DOY =',stop.timetuple().tm_yday)

Data start DOY = 81
Data end DOY = 85


In [9]:
# Create a pysat instrument for Kp data from the pysatSpaceweather package
kp_his = pysat.Instrument(inst_module=py_sw.instruments.sw_kp, tag='def', update_files=True)

# Set the type of orbit delimiting for the COSMIC 2 instrument
orbit_info = {'index': 'sclongitudeAVG', 'kind': 'longitude'}

# Create COSMIC 2 ion velocity meter instrument
saber = pysat.Instrument(platform='timed', name='saber', orbit_info=orbit_info)

In [10]:
# Download the data (only needed once)
#saber.download(start, stop)
#kp_his.download(start, stop)

In [11]:
# Load the Kp and COSMIC 2 data
kp_his.load(2023, 81, end_yr=2023, end_doy=85)
saber.load(2023, 81, end_yr=2023, end_doy=85)

/srv/conda/envs/notebook/lib/python3.10/site-packages/pysat/_instrument.py:2388: FutureWarning: The return type of `Dataset.dims` will be changed to return a set of dimension names in future, in order to be more consistent with `DataArray.dims`. To access a mapping from dimension names to lengths, please use `Dataset.sizes`.
  if new_data[idat].dims != new_data[idat + 1].dims:
/srv/conda/envs/notebook/lib/python3.10/site-packages/pysat/utils/coords.py:311: FutureWarning: The return type of `Dataset.dims` will be changed to return a set of dimension names in future, in order to be more consistent with `DataArray.dims`. To access a mapping from dimension names to lengths, please use `Dataset.sizes`.
  dims = [dim_key for dim_key in list(sdata.dims.keys())
/srv/conda/envs/notebook/lib/python3.10/site-packages/pysat/utils/coords.py:314: FutureWarning: The return type of `Dataset.dims` will be changed to return a set of dimension names in future, in order to be more consistent with `DataArr

In [ ]:
# Here we'll get a little information about the TIMED SABER data we're looking at

# Basic info about the mission and instrument with some links to more information
print(saber.meta['Project'])
print(saber.meta['Descriptor'])
print(saber.meta['Link_title'])
print(saber.meta['HTTP_link'])

# List of all of the variables in the loaded data
list(saber.data.variables)

In [ ]:
# Now let's look at the orbit of the spacecraft

fig, axs = plt.subplots(3, 1)
# TIMED has a mostly circular orbit
saber['scaltitudeAVG'].plot(ax=axs[0])
axs[0].set_xlim(start, start+dt.timedelta(minutes=90))
axs[0].set_title(saber.meta['scaltitudeAVG']['standard_name'])

# The SABER instrument is a radiometer that scans the Earth's Limb
saber['tpaltitude'][:, 50].plot(ax=axs[1])
axs[1].set_xlim(start, start+dt.timedelta(minutes=90))
axs[1].set_title(saber.meta['tpaltitude']['standard_name'])

# TIMED has an inclination of about 75 degrees 
saber['sclatitudeAVG'].plot(ax=axs[2])
axs[2].set_xlim(start, start+dt.timedelta(minutes=90))
axs[2].set_title(saber.meta['sclatitudeAVG']['standard_name'])
plt.tight_layout()

In [ ]:
"""The SABER instrument on the TIMED satellite makes scans of the Earth's Limb with a radiometer.
So the measurements of neutral density, pressure, and temperature are not being made in-situ, or at the satellite.
Instead, they're being made over the horizon from the satellite's point of view.
This diagram depicts the configuration of the satellite and the radiometer view.
By scanning the limb, SABER gets a vertical profile of the measured quantities from about 3km to 200km.
SABER has 10 channels for different wavelengths that correspond to different neutral species.
"""

from IPython.display import Image
Image("timed_view.png")

In [ ]:
saber_time = saber['time']
X = [saber_time, saber_time]
x = np.repeat(X, 200, axis=0).transpose()
Z = np.log10(saber['ktemp'])

# Plot the ion density and Kp
fig, axs = plt.subplots(2, 1)
pcm = axs[0].pcolormesh(x, saber['tpaltitude'], Z, shading='nearest', cmap='plasma')
axs[0].set_xlim(start, stop)
axs[0].set_ylim(0, 120)
cbar = fig.colorbar(pcm, ax=axs[0], location='top')
cbar.set_label(label='neutral density log10(cm^-3)')
kp_his['Kp'].plot(ax=axs[1])
axs[1].set_xlim(pd.Timestamp(start), pd.Timestamp(stop))
plt.tight_layout()

In [ ]:
# But you don't see much if any response in the temperature to the storm so we'll plot a single channel
# Also we'll smooth it out a little and see if that makes the temperature response any clearer
print(saber['tpaltitude'][:, 137].mean())
print(saber['time'][1] - saber['time'][0])
saber['ktemp'][:, 137].plot(label='Temperature K')
avg_temp = saber['ktemp'][:, 137].rolling(time=240, center=True).mean()
avg_temp.plot(label='~4 hour rolling average')
plt.legend()

In [ ]:
# Now lets look at the data from Kamodo's perspective. First inspect the size and content structure
nTimes = saber['tpaltitude'].shape[0]
nChans = saber['tpaltitude'].shape[1]
print('Data array shape =',saber['tpaltitude'].shape)

In [ ]:
# What's packed in it
saber['tpaltitude'][0,0]

In [ ]:
# Lets functionalize it in Kamodo and take a quick look.
coord_dict = {'time': {'units': 'hr', 'data': np.linspace(0., 96., nTimes)},
              'channel': {'units': '', 'data': np.linspace(0, 399, nChans)}}
var_dict = {'alt': {'units': 'km', 'data': saber['tpaltitude'].values},
            'lon': {'units': 'deg', 'data': saber['tplongitude'].values},
            'lat': {'units': 'deg', 'data': saber['tplatitude'].values},
            'temp': {'units': 'K', 'data': saber['ktemp'].values}}
ko = FD(coord_dict, var_dict)
ko

In [ ]:
var = 'alt'
figA = ko.plot(var, plot_partial={var: {'channel': 137}})
figA

In [ ]:
# It's not a big deal, but lets quick convert the X axis to datetime rather than hours from start of data
newdt = [pytz.utc.localize(datetime.utcfromtimestamp(v)) 
         for v in pytz.utc.localize(start).timestamp()+3600.*figA.data[0].x]
figA.data[0].x = newdt
figA

In [ ]:
# I can analyze for a specific channel, but as you see the altitude varies a bit.
# Lets do a quick little interpolation to extract position and values at exactly 100km

# Set the height to interpolate to, then build interpolation factors
h = 107.
mF = np.ndarray(shape=(nTimes,2), dtype=np.float32)
mI = np.ndarray(shape=(nTimes,2), dtype=int)
for i in range(nTimes):
    talt = saber['tpaltitude'].values[i,:]
    dif = np.absolute(talt - h)
    mI[i,0] = dif.argmin()
    if talt[mI[i,0]] >= h:
        mI[i,1] = mI[i,0]+1
    else:
        mI[i,1] = mI[i,0]-1
    mF[i,1] = (talt[mI[i,0]]-h)/(talt[mI[i,0]]-talt[mI[i,1]])
    mF[i,0] = 1. - mF[i,1]

In [ ]:
# Set the satellite values to interpolate from and compute the interpolation
v1 = saber['tpaltitude'].values
v2 = saber['tplongitude'].values
v3 = saber['tplatitude'].values
v4 = saber['density'].values
v5 = saber['ktemp'].values
valt = np.ndarray(shape=(nTimes), dtype=np.float32)
vlon = np.ndarray(shape=(nTimes), dtype=np.float32)
vlat = np.ndarray(shape=(nTimes), dtype=np.float32)
vden = np.ndarray(shape=(nTimes), dtype=np.float32)
vtem = np.ndarray(shape=(nTimes), dtype=np.float32)
for i in range(nTimes):
    valt[i] = (v1[i,mI[i,0]]*mF[i,0]) + (v1[i,mI[i,1]]*mF[i,1])
    vlon[i] = (v2[i,mI[i,0]]*mF[i,0]) + (v2[i,mI[i,1]]*mF[i,1])
    vlat[i] = (v3[i,mI[i,0]]*mF[i,0]) + (v3[i,mI[i,1]]*mF[i,1])
    vden[i] = (v4[i,mI[i,0]]*mF[i,0]) + (v4[i,mI[i,1]]*mF[i,1])
    vtem[i] = (v5[i,mI[i,0]]*mF[i,0]) + (v5[i,mI[i,1]]*mF[i,1])

In [ ]:
# Time is a little more complicated due to datetime64
vtime = saber['tplatitude'].time.values
vtimets = ((vtime - np.datetime64('1970-01-01T00:00:00')) / np.timedelta64(1, 's'))
vtimedt = np.array([pytz.utc.localize(datetime.utcfromtimestamp(tt)) for tt in vtimets])

In [ ]:
# The interpolator needs longitude +/- 180, not 0 to 360
lmask = vlon > 180.
vlon[lmask] -= 360.

In [ ]:
# Now, I'm saving the time and postion I want to look at to a file to use for interpolation
#   from a TIEGCM run to compare the model values with the observed data
with open("Extraction_Path.txt", "w") as f:
    f.write("#Model files used:\tunknown\n")
    f.write("#Model used:\tTIEGCM\n")
    f.write("#Coordinates:\tGDZ\tsph\n")
    f.write("#utc_time\tc1\tc2\tc3\n")
    f.write("#[s]\t[R_E]\t[R_E]\t[R_E]\n")
    for i in range(nTimes):
        f.write("%f\t%f\t%f\t%f\n" % (vtimets[i], vlon[i], vlat[i], valt[i]))

In [ ]:
# The file looks like this
!head -10 Extraction_Path.txt

In [ ]:
# Lets functionalize this new data in Kamodo.
coord_dict = {'time': {'units': 'hr', 'data': np.linspace(0., 96., nTimes)}}
var_dict = {'alt': {'units': 'km', 'data': valt},
            'lon': {'units': 'deg', 'data': vlon},
            'lat': {'units': 'deg', 'data': vlat},
            'density': {'units': 'deg', 'data': vden},
            'temp': {'units': 'K', 'data': vtem}}
ko2 = FD(coord_dict, var_dict)
ko2

In [ ]:
# Make a quick plot to see that all is well.
ko2.plot('alt', 'lon', 'lat', 'temp')

In [ ]:
# Let's load Kamodo's TIEGCM reader and set some variables

model = 'TIEGCM'
variables_requested = ['T_n', 'rho']  # one variable from each coordinate
# change file path to where data is stored on your machine
file_dir = '/home/jovyan/scratch_space/Kamodo_Model_Output/TIEGCM/TIEGCM-Weimer-02_2023-03-TP-01_012624_IT_4/'

reader = MW.Model_Reader(model)

In [ ]:
# What density and temperature values are in the model output?
MW.Variable_Search('density', model, file_dir)
print(' ')
MW.Variable_Search('temperature', model, file_dir)

In [ ]:
# Load the model output into Kamodo
kamodo_object = reader(file_dir, variables_requested=variables_requested)
kamodo_object

In [ ]:
# What's the base date for what we've just read in
kamodo_object.filedate

In [ ]:
# Generate a plot to see what the model provides
toColor(kamodo_object.plot('T_n_ijk', plot_partial={'T_n_ijk': {'time': 74., 'height': h}}), colorscale="Viridis")

In [ ]:
# We have special extras for nicer plots
plot_date = dt.datetime(2023, 3, 22, 2).replace(tzinfo=dt.timezone.utc)
pf.GDZSlice4D(kamodo_object.T_n, 'T_n[K]', 'TIEGCM', plot_date, 'Lon-Lat', fixed_alt=h, fixed_time=74.,
             plotCoord='GEO', shoreline=True, 
              title='Upgraded 2D plot with shorelines')

In [ ]:
# Set some flythrough extraction variables
start_utcts = dt.datetime(2023, 3, 22, 0).replace(tzinfo=dt.timezone.utc).timestamp()
end_utcts = dt.datetime(2023, 3, 26, 0).replace(tzinfo=dt.timezone.utc).timestamp()-1
variables = ['T_n', 'rho', 'TEC']  # one or more variable names to retrieve

In [ ]:
# DON'T EXECUTE
# This is for example only. If data was at the satellite location, this will extract values there automatically
#
#dataset = 'timed' 
#coord_type = 'GEO'  # Desired coordinate system for retrieved trajectory.
#plot_coord = 'GSE'  # Coordinate system chosen for output plots
## Choose naming convention for output files
#output_name = 'TIEGCM_timed_output.txt' 
#!rm -f TIEGCM_timed_output*

In [ ]:
# DON'T EXECUTE
#
#results_real = SF.RealFlight(dataset, start_utcts, end_utcts, model, file_dir, variables, 
#                             coord_type=coord_type, output_name=output_name, plot_coord=plot_coord)
#kamodo_object_real = SF.O.Functionalize_SFResults(model, results_real)
#kamodo_object_real

In [ ]:
# Now we will extract along 107km altitude path from the file we saved above and load into Kamodo
pfile = 'Extraction_Path.txt'
results_my = SF.MyFlight(pfile, model, file_dir, variables)
kamodo_object_my = SF.O.Functionalize_SFResults(model, results_my)
kamodo_object_my

In [ ]:
# Plot the neutral temperature
figT = kamodo_object_my.plot('T_n')
figT

In [ ]:
# Now take the observed values, plot them, adjust the X axis time, and add to the previous plot
figS = ko2.plot('temp')
newdt = [pytz.utc.localize(datetime.utcfromtimestamp(v)) 
         for v in pytz.utc.localize(start).timestamp()+3600.*figS.data[0].x]
figS.data[0].x = newdt
figT.add_trace(figS.data[0])

In [ ]:
# simple function to reduce resolution by 2 via averaging.
def halfResolution(inFig):
    inX = inFig.data[0]['x']
    inY = inFig.data[0]['y']
    outX, outY = [], []
    for i in range(len(inX)):
        if (i % 2) == 1:
            aveX = inX[i-1] + (inX[i]-inX[i-1])/2. # This works for datetimes and floats
            aveY = inY[i-1] + (inY[i]-inY[i-1])/2.
            outX.append(aveX)
            outY.append(aveY)
    inFig.data[0]['x'] = outX
    inFig.data[0]['y'] = outY
    name = inFig.data[0]['name']+'.'
    inFig.data[0]['name'] = name

    return inFig

In [ ]:
halfResolution(figS)
figT.add_trace(figS.data[0])

In [ ]:
from kamodo_ccmc.flythrough.plots import SatPlot4D

In [ ]:
SatPlot4D('T', vtimets, vlon, vlat, valt, vtem, 'K', 'GDZ', 'sph', 'GEO', 
          'all', 'TIMED-SABER 107km', type='2D')

In [ ]:
SatPlot4D('T', vtimets, vlon, vlat, valt, vtem, 'K', 'GDZ', 'sph', 'GEO', 
          'all', 'TIMED-SABER 107km', type='2DLT')

In [ ]:
SatPlot4D('T', vtimets, vlon, vlat, valt, vtem, 'K', 'GDZ', 'sph', 'GSE', 
          'orbitE', 'TIMED-SABER 107km', type='3D')#, body='earth')